In [52]:
import numpy as np
from time import time
from joblib import Parallel, delayed

import numba
from numba import jit

In [62]:
def very_math(x):
     return np.sum([np.sin(np.log(i)*x) for i in range(2, 200)], axis=0)
    
data = np.linspace(0, 1, 10000)

def do_job_serial(data):
    N = data.shape[0]
    rez = np.zeros(N)
    for j in range(N):
        rez[j] = very_math(data[j])
    return rez

%timeit very_math(1)

t = time()
results_serial = do_job_serial(data)
print("Serial time", time()-t)

366 µs ± 2.55 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Serial time 3.3286476135253906


## Joblib

AKA complete junk

In [50]:
t = time()
results_joblib = Parallel(n_jobs=-1, backend="threading")(map(delayed(very_math), data))
print("Parallel time", time()-t)

print("Difference:", np.linalg.norm(results_joblib-results_serial))

Parallel time 6.15205192565918
Difference: 0.0


## Numba

It works! Just need to convert all functions to numba, and avoid using lists.

In [65]:
@jit(nopython=True)
def very_math_numba(x):
    rez = 0
    for i in range(2, 200):
        rez += np.sin(np.log(i)*x)
    return rez
    
@jit(nopython=True)
def do_job_numba(data):
    N = data.shape[0]
    rez = np.zeros(N)
    for j in range(N):
        rez[j] = very_math_numba(data[j])
    return rez
    
%timeit very_math_numba(1)
    
t = time()
results_numba = do_job_numba(data)
print("Parallel time", time()-t)

print("Difference:", np.linalg.norm(results_numba-results_serial))

6.99 µs ± 130 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Parallel time 0.14512038230895996
Difference: 4.851862508105707e-12


In [53]:
print(numba.__version__)

0.42.0
